In [18]:
# ###############################################################
#       d888888b                         d888888b
#    d888    8888b                    d888888   888b
#  d88    88  898888b               d8888  888     88b
# d8P        88888888b             d88888888888     b8b
# 88        8888888888             88888888888       88
# 88       88888888888             8888888888        88
# 98b     88888888888P             988888888        d8P
#  988     888  8888P      _=_      9888898  88    88P
#    9888   888888P      q(-_-)p       98888    888P
#       9888888P         '_) (_`         9888888P
#          88            /__/  \            88
#          88          _(<_   / )_          88
#         d88b        (__\_\_|_/__)        d88b
###
###                  佛祖保佑，Bug退去；
###                  阿弥陀佛，善哉善哉
# ###############################################################

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import time
import math

%matplotlib inline

In [19]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

ValueError: list.remove(x): x not in list

In [20]:
### 图像预处理
def preprocess(file_name):
    img = cv2.imread(file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    chs = cv2.split(img)
    hue = ((chs[0] > 30) * 1) * chs[0]
    res = ((hue < 50) * 1) * hue

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    res = cv2.morphologyEx(res.astype(np.float32), cv2.MORPH_ERODE, kernel)
    resize = cv2.resize(res, (196, 196))
    resize = resize.reshape((1, 196, 196))
    resize = cv2.normalize(resize, resize, 0, 255, cv2.NORM_MINMAX)

#     print(file_name)
#     cv2.imwrite("test.jpg", resize[0])

    return resize

In [29]:
train_path = '/media/lor/data/data/plant-seedlings-classification/train'
train_samples = []
train_labels = []
label_to_idx = {}
for (i, cat) in enumerate(os.listdir(train_path)):
    label_to_idx[cat] = i
    cat_path = os.path.join(train_path, cat)
#     print(cat_path)
    for item in os.listdir(cat_path):
        train_labels.append(i)
        train_samples.append(os.path.join(cat_path, item))
# print(len(train_labels))
# print(len(train_samples))
train_labels.extend(train_labels)
train_samples.extend(train_samples)

In [30]:
class PlantDS(torch.utils.data.Dataset):
    def __init__(self, samples, targets):
        self.samples = samples
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        target = self.targets[idx]
        sample = preprocess(self.samples[idx])
        if idx >= 4750:
            sample = cv2.flip(sample, 0)
        sample = sample / 255
        return (target, sample)

In [31]:
train_ds = PlantDS(train_samples, train_labels)
indices = list(range(train_ds.__len__()))
np.random.shuffle(indices)
train_indices = indices[0:9000]
valid_indices = indices[9000:9500]
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_indices)

In [32]:
train_dloader = torch.utils.data.DataLoader(train_ds, batch_size=8, num_workers=4, sampler=train_sampler)
valid_dloader = torch.utils.data.DataLoader(train_ds, batch_size=1, num_workers=4, sampler=valid_sampler)

In [33]:
class PlantNet(nn.Module):
    def __init__(self):
        super(PlantNet, self).__init__()
        self.layer_1 = nn.Sequential(
                        nn.Conv2d(1, 8, kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(8),
                        nn.LeakyReLU(),
                        nn.Conv2d(8, 12, kernel_size=(5, 5), stride=1, padding=2),
                        nn.BatchNorm2d(12),
                        nn.LeakyReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2),
        )
        self.layer_2 = nn.Sequential(
                        nn.Conv2d(12, 18, kernel_size=(5, 5), stride=1, padding=1),
                        nn.BatchNorm2d(18),
                        nn.LeakyReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2),
                        nn.Conv2d(18, 24, kernel_size=(5, 5), stride=1, padding=2),
                        nn.BatchNorm2d(24),
                        nn.LeakyReLU(),
                        nn.MaxPool2d(kernel_size=(3, 3), stride=3),
        )
        
        self.layer_exp_1 = nn.Sequential(
                        nn.Conv2d(24, 48, kernel_size=(3, 3), stride=1, padding=1),
                        nn.BatchNorm2d(48),
                        nn.LeakyReLU(),
                        nn.MaxPool2d(kernel_size=(2, 2), stride=2),
        )
        
        self.layer_3 = nn.Linear(8 * 8 * 48, 1024)
        self.layer_4 = nn.Linear(1024, 341)
#         self.layer_5 = nn.Linear(341, 113)
        self.layer_6 = nn.Linear(341, 12)
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_exp_1(out)
        out = out.reshape(out.shape[0], -1)
        out = self.layer_3(out)
        out = self.layer_4(F.relu(out))
#         out = self.layer_5(F.relu(out))
        out = self.layer_6(F.relu(out))

        return out

In [34]:
num_epochs = 10
net = PlantNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
total_step = len(train_dloader)

In [35]:
net.train()
start = time.time()
for epoch in range(num_epochs):
    for i, (target, sample) in enumerate(train_dloader):
        pred = net(sample)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:

            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
end = time.time()
interval = end - start
print("Train spending: {}:{}".format(math.floor(interval / 60), interval - math.floor(interval / 60) * 60))

Epoch [1/10], Step [100/1125], Loss: 1.1408
Epoch [1/10], Step [200/1125], Loss: 1.1259
Epoch [1/10], Step [300/1125], Loss: 1.9303
Epoch [1/10], Step [400/1125], Loss: 2.2203
Epoch [1/10], Step [500/1125], Loss: 1.7562
Epoch [1/10], Step [600/1125], Loss: 1.4677
Epoch [1/10], Step [700/1125], Loss: 1.2713
Epoch [1/10], Step [800/1125], Loss: 1.7741
Epoch [1/10], Step [900/1125], Loss: 0.8069
Epoch [1/10], Step [1000/1125], Loss: 1.1476
Epoch [1/10], Step [1100/1125], Loss: 1.4585
Epoch [2/10], Step [100/1125], Loss: 1.0584
Epoch [2/10], Step [200/1125], Loss: 0.5797
Epoch [2/10], Step [300/1125], Loss: 0.6667
Epoch [2/10], Step [400/1125], Loss: 1.1067
Epoch [2/10], Step [500/1125], Loss: 0.9751
Epoch [2/10], Step [600/1125], Loss: 0.8032
Epoch [2/10], Step [700/1125], Loss: 0.6883
Epoch [2/10], Step [800/1125], Loss: 1.7047
Epoch [2/10], Step [900/1125], Loss: 0.6385
Epoch [2/10], Step [1000/1125], Loss: 0.7781
Epoch [2/10], Step [1100/1125], Loss: 0.3574
Epoch [3/10], Step [100/1125

In [36]:
start = time.time()
net.eval()
total = 0
for (target, sample) in valid_dloader:
    pred = net(sample)
    total = total + (torch.argmax(pred, 1) == target).sum().item()

end = time.time()
interval = end - start
print("Accuracy: {0}%".format(100 * total / len(valid_indices)))
print("Test spending: {}".format(interval))

Accuracy: 92.0%
Test spending: 6.253731727600098
